In [4]:
#from sequences import *
from extraction_utils import extract
from get_data_v2 import *
import re
import numpy as np
import os
import glob
import warnings
from copy import deepcopy
import json
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'extraction_utils'

In [6]:

def find_json_file(directory_path):
    # Use glob to find all JSON files in the directory
    json_files = glob.glob(os.path.join(directory_path, "*.json"))
    
    # Check if any JSON files were found
    if json_files:
        return json_files[0]
    else:
        return None

In [7]:
def check_block_ended_correctly(data):
    experiment_results = {}
    for block_key, block_data in data.items():
        if block_key.startswith("Block_"):
            experiment_results[block_key] = block_data.get("Experiment ended correctly", None)
    return experiment_results

In [8]:
#analog_in et digital_in
ANALOG_TRIGGERS_MAPPING = {"MAIN": 1, "PLAYBACK": 0, "MOCK": 3, "TARGET": 2}
#ANALOG_TRIGGERS_MAPPING = {"MAIN": 1, "PLAYBACK": 0}

#hje checke que je suis en v2 dans le json
path='/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240521_SESSION_02/headstage_0/'

json_file_path = find_json_file(path)

with open(json_file_path, 'r') as f:
        json_file = json.load(f)

blocks_ended_correctly = list(check_block_ended_correctly(json_file))

dig_trig_mapping = get_digital_mapping(json_file)


# for le nombre de blocks
n_blocks = get_n_iter(json_file) #ne marche pas encore
# pour un block :
n_blocks = 3


#extraire les canaux analogiques et digitaux avec le bon mapping des canaux digitaux
#triggers = extract(path)

#extraire positions et tons block par block
positions_tr, tones_tr, positions_pb, tones_pb, tones_mck = [], [], [], [], []
for block in range(0, n_blocks):
    print(block)
    if blocks_ended_correctly[block]:
        
        # extraire les positions et les tons
        tones_tr.append(get_tones(path, json_file, ['tracking', 'playback', 'mock'], f"Block_00{block}")['tracking'][0])
        positions_tr.append(get_positions(path, json_file, ['tracking', 'playback'], f"Block_00{block}")['tracking'][0])
        #tones = extract_tones_path(json_file, block, condition)


        #en playback : convertir les positions en tones pour avoir les MOCK
        # mock = get_tones_from_position (nom donné au pif)
        tones_pb.append(get_tones(path, json_file, ['tracking', 'playback', 'mock'], f"Block_00{block}")['playback'][0])
        positions_pb.append(get_positions(path, json_file, ['tracking', 'playback'], f"Block_00{block}")['playback'][0])
        tones_mck.append(get_tones(path, json_file, ['tracking', 'playback', 'mock'], f"Block_00{block}")['mock'][0])
    else:
        break

NameError: name 'glob' is not defined

In [8]:
# découper les triggers en bloc

#extraire les canaux analogiques et digitaux avec le bon mapping des canaux digitaux
triggers = extract(path)
analog_channels = triggers['ANALOG']
digital_channels = triggers['DIGITAL']

tr_trigs = analog_channels['MAIN']
pb_trigs = analog_channels['PLAYBACK']
mck_trigs = analog_channels['MOCK']



    
    

/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240521_SESSION_02/headstage_0/trig_dig_chan_{}.npy
[False False False ... False False False]


In [39]:
def associate_tones_and_triggersv2(tones, triggers, condition):
    """""
    Dans le cas où on a les mocks triggers
    
    """
    # Create an empty dictionary to store associations
    tt = {}
    if condition == 'tracking':
        cond = 'tr'
    elif condition == 'playback':
        cond = 'pb'
    elif condition == 'mock':
        cond = 'mk'
    # Iterate over each index and subarray of tones
    for i, block_tones in enumerate(tones): 
        # Get the length of the current subarray
        subarray_length = len(block_tones)
        
        # Take the corresponding triggers for the current subarray
        subarray_triggers = triggers[:subarray_length]
        
        # Associate triggers with tones in a dictionary
        tt[f"{cond}_{i}"] = [block_tones, subarray_triggers]
        
        # Remove the triggers used for this subarray
        triggers = triggers[subarray_length:]
    return tt


In [40]:
def extract_recording_length(path):
    file = path+'/recording_length.bin'
    with open(file, 'rb') as file:
            recording_length = file.read()
    recording_length = recording_length.decode('utf-8')

        # Extract only the numbers using a simple filter
    recording_length = int(''.join(filter(str.isdigit, recording_length)))
    return int(recording_length)

In [13]:
def create_tt(path, analog_channels, tones_tr, tones_pb):
    """"
    Fonction ok dans le cas où on a pas de mock triggers 
    
    """
    n_iter = len(tones_pb)
    tr_trigs = analog_channels['MAIN']
    pb_trigs = analog_channels['PLAYBACK']
    
    # Assuming you have functions to create the arrays 'order', 'n_iter', 'recording_length', 'keys', and 'numbers'
    # Assuming you have a function to extract recording length from the path
    recording_length = extract_recording_length(path)
    
    # Create the arrays for 'pb_' and 'tr_' using the provided function
    tt_tr = associate_tones_and_triggersv2(tones_tr, tr_trigs, 'tracking')
    tt_pb = associate_tones_and_triggersv2(tones_pb, pb_trigs, 'playback')
    
    # Combine 'tt_tr' and 'tt_pb' dictionaries
    tt_tr.update(tt_pb)
    
    # Create the final object
    tt_object = {
        'n_iter': n_iter,
        'recording_length': recording_length,
        **tt_tr  # Add the combined triggers and tones dictionaries
    }
    
    return tt_object


In [41]:
def create_tt(path, analog_channels, tones_tr, tones_pb, tones_mck):
    """"
    Dans le cas où on a les mock triggers 
    
    """
    n_iter = len(tones_pb)
    tr_trigs = analog_channels['MAIN']
    pb_trigs = analog_channels['PLAYBACK']
    mck_trigs = analog_channels['MOCK']
    
    # Assuming you have functions to create the arrays 'order', 'n_iter', 'recording_length', 'keys', and 'numbers'
    # Assuming you have a function to extract recording length from the path
    recording_length = extract_recording_length(path)
    
    # Create the arrays for 'pb_' and 'tr_' using the provided function
    tt_tr = associate_tones_and_triggersv2(tones_tr, tr_trigs, 'tracking')
    tt_pb = associate_tones_and_triggersv2(tones_pb, pb_trigs, 'playback')
    tt_mck = associate_tones_and_triggersv2(tones_mck, mck_trigs, 'mock')
    
    # Combine 'tt_tr' and 'tt_pb' dictionaries
    tt_tr.update(tt_pb)
    tt_tr.update(tt_mck)
    
    # Create the final object
    tt_object = {
        'n_iter': n_iter,
        'recording_length': recording_length,
        **tt_tr  # Add the combined triggers and tones dictionaries
    }
    
    return tt_object

In [42]:
tt = create_tt(path, analog_channels, tones_tr, tones_pb, tones_mck)


In [43]:
for key, value in tt.items():
    print(key)

n_iter
recording_length
tr_0
tr_1
tr_2
pb_0
pb_1
pb_2
mk_0
mk_1
mk_2


In [38]:
print(tt['mck_2'])

[array([ 533.,  460.,  533.,  617.,  533.,  617.,  533.,  617.,  533.,
        617.,  533.,  617.,  533.,  617.,  715.,  617.,  715.,  617.,
        715.,  828.,  959.,  828.,  959.,  828.,  959.,  828.,  715.,
        617.,  533.,  460.,  397.,  460.,  533.,  460.,  533.,  460.,
        533.,  460.,  533.,  617.,  715.,  828.,  959., 1111., 1287.,
       1490., 1727., 1490., 1287., 1111., 1287., 1490., 1727., 2000.,
       2317., 2684., 3109., 3601., 4172., 2684., 3601., 2684., 3109.,
       5598., 4172., 1111.,  959., 1111., 1287., 1490., 1287., 1111.,
        959.,  828.,  715.,  828.,  715.,  828.,  959., 1111., 1287.,
       1490., 1727., 2000., 2317.,  959., 2000., 1111., 1287., 1111.,
        959., 1111.,  715.,  617.,  533.,  715.,  828.,  715.,  828.,
        715.,  828.,  959.,  828.,  959.,  828.,  959.,  828.,  959.,
        828.,  959.,  828.,  959.,  828.,  959.,  828.,  959.,  828.,
        959., 1111., 1287., 1490., 1287., 1111.,  959.,  828.,  715.,
        617.,  715.

In [44]:
np.savez(path+'tt.npz', tt)

voila c'est tout depuis le debut pour faire le fichier tt quand il n'y a pas de mock triggers !

In [147]:
import numpy as np
file_path = '/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/BURRATA/BURRATA_20240503_SESSION_02/headstage_1/tt.npz'
#file_path = '/mnt/working2/felicie/Python_theremin/Analyse/Analyse/Experiment/FRINAULT/FRINAULT_20230223/tt.npz'
def read_npz_file(file_path):
    format_tt = []
    # Load the .npz file
    npz_file = np.load(file_path, allow_pickle=True)
    
    # Access the arrays stored in the .npz file
    for array_name in npz_file:
        array_data = npz_file[array_name]
        format_tt.append(array_data)
    return format_tt
        

# Example usage:

mytt = read_npz_file(file_path)
